In [1]:
from readjson import process_json_file
sentences = process_json_file("raw/ultrachat_release_230407.json")

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class SkipGramModel(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super(SkipGramModel, self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, emb_dim) # center embedding
        self.embedding_u = nn.Embedding(vocab_size, emb_dim) # out embedding
        self.logsigmoid = nn.LogSigmoid()

    def forward(self, center_words, target_words, outer_words):
        center_embeds = self.embedding_v(center_words) # B x 1 x D
        target_embeds = self.embedding_u(target_words) # B x 1 x D
        outer_embeds = self.embedding_u(outer_words) # B x V x D

        scores = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # Bx1xD * BxDx1 => Bx1
        norm_scores = outer_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # BxVxD * BxDx1 => BxV

        loss = self.logsigmoid(scores) + torch.sum(self.logsigmoid(-1*norm_scores), dim=1)
        return -1*loss

In [3]:
model = SkipGramModel(len(sentences), 100)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

: 

: 